In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import SpectralClustering, KMeans

In [2]:
def rbf_kern(v1, v2, scale):
    """
    radial-basis function
    """
    return np.exp(la.norm(v1-v2, 2)**2 / -scale)

In [3]:
def form_wgt_mat(A, kern, gamma, tol=1e-10):
    """
    generate weight matrix
    INPUT:
    pim vectors A
    kernel function
    tolerance - any weight below this value will be set to zero
    """
    (r, c) = A.shape
    wgt_mat = np.zeros(r * r).reshape(r, r)

    for i in range(r):
        ### progress bar ###
        pb = "~"*int(i/r*100)+" "*int((1-i/r)*100)+"|"
        print(pb, end="\r")
        ####################
        for j in range(r):
            if i == j: wgt_mat[i, j] = 0
            else: wgt_mat[i, j] = rbf_kern(A[i, :], A[j, :], gamma)

    for i in range(r):
        for j in range(r):
            if wgt_mat[i, j] < tol and wgt_mat[i, j] != 0:
                wgt_mat[i, j] = 0

    return wgt_mat

In [4]:
def cluster_composition(clabs, df, idcol):
    """
    return percentage of each gesture composing each cluster
    INPUT:
    clabs - cluster labels (numpy array)
    df - data frame of observations including column of IDs (pandas DataFrame)
    idcol - specific column which includs ID values
    """
    df["cluster_ID"] = clabs
    unq_cats = df[idcol].unique() # unique category variables in idcol

In [5]:
pim_df = pd.read_csv("./pim_vectors_mp40.csv")
pim_vecs = pim_df.values[:, :-2]

W = form_wgt_mat(pim_vecs, rbf_kern, 15)
D = np.diag(W.sum(axis=1))
L = D - W # graph laplacian

evals, evecs = la.eig(L)
eidx = np.argsort(evals.real)
evecs = evecs.real[:, eidx]
evals = evals.real[eidx]

In [6]:
X = evecs[:, :4]

In [7]:
kmeans = KMeans(n_clusters=4, precompute_distances=True)
kmeans.fit_predict(X)
clabs = kmeans.labels_
idcol = "gest"

In [11]:
tstfrm = pim_df
tstfrm["cluster_ID"] = clabs
unq_cats = tstfrm[idcol].unique() # unique category variables in idcol
unq_cats = unq_cats.astype("int")
unq_cids = tstfrm["cluster_ID"].unique() # unique cluster IDs

In [12]:
comp_df = np.zeros(len(unq_cats)*len(unq_cids)).reshape(len(unq_cids), -1)

for n, i in enumerate(unq_cids):
    clust_tot = sum(tstfrm.cluster_ID==i)
    temp = tstfrm[idcol][tstfrm.cluster_ID==i]
    for m, j in enumerate(unq_cats):
        comp_df[n, m] = (sum(temp == j) / clust_tot)*100
    

In [13]:
cpdf = pd.DataFrame(comp_df)
cpdf.columns = unq_cats
cpdf.index = unq_cids
cpdf

,5,3,4,2,1,6
0,16.744186,16.744186,16.744186,16.744186,16.27907,16.744186
2,0.000000,0.000000,0.000000,0.000000,100.00000,0.000000
3,0.000000,0.000000,0.000000,0.000000,100.00000,0.000000
1,0.000000,0.000000,0.000000,0.000000,100.00000,0.000000
